# All Model saves here

## import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, random_split
import DeepMIMOv3
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import time


plt . rcParams [ 'figure.figsize' ]  =  [ 12 ,  8 ]  # 기본 플롯 크기 설정

ModuleNotFoundError: No module named 'torch'

In [2]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.4 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 6.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 14.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 MB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 4.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━

## GPU Settings

In [ ]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import torch
print(torch.version.cuda)                   
print(torch.backends.cudnn.version())       
print("CUDA available:", torch.cuda.is_available())  # True

## DeepMIMOv3 dataset

In [ ]:
parameters = DeepMIMOv3.default_params()

In [ ]:
## Change parameters for the setup
# Scenario O1_60 extracted at the dataset_folder
#LWM dynamic senario
# parameters['dataset_folder'] = r'/content/drive/MyDrive/Colab Notebooks/LWM'
scene = 15 # scene 15
# change my linux route
parameters['dataset_folder'] = '/home/dlghdbs200/LWM'

# scnario = 02_dyn_3p5 <- download file
parameters['scenario'] = 'O2_dyn_3p5'
parameters['dynamic_scenario_scenes'] = np.arange(scene) #scene 0~9

# Up to 10 multipath paths per user-to-base station channel
parameters['num_paths'] = 10

# User rows 1-100
parameters['user_rows'] = np.arange(100)
# User subsampling
parameters['user_subsampling'] = 0.01

# Activate only the first basestation
parameters['active_BS'] = np.array([1])

parameters['activate_OFDM'] = 1

parameters['OFDM']['bandwidth'] = 0.05 # 50 MHz
parameters['OFDM']['subcarriers'] = 512 # OFDM with 512 subcarriers
parameters['OFDM']['selected_subcarriers'] = np.arange(0, 64, 1)
#parameters['OFDM']['subcarriers_limit'] = 64 # Keep only first 64 subcarriers

parameters['ue_antenna']['shape'] = np.array([1, 1]) # Single antenna
parameters['bs_antenna']['shape'] = np.array([1, 32]) # ULA of 32 elements
#parameters['bs_antenna']['rotation'] = np.array([0, 30, 90]) # ULA of 32 elements
#parameters['ue_antenna']['rotation'] = np.array([[0, 30], [30, 60], [60, 90]]) # ULA of 32 elements
#parameters['ue_antenna']['radiation_pattern'] = 'isotropic'
#parameters['bs_antenna']['radiation_pattern'] = 'halfwave-dipole'

In [ ]:
## dataset setting (chunked on‑the‑fly generation)
import time, gc
from tqdm import tqdm

# 0~999 scene index , process 50 at that time
scene_indices = np.arange(scene)
chunk_size   = 5
all_data     = []

# Call generate_data for each scene chunk
for i in tqdm(range(0, len(scene_indices), chunk_size)):
    chunk = scene_indices[i : i+chunk_size].tolist()
    parameters['dynamic_scenario_scenes'] = chunk

    start = time.time()
    data_chunk = DeepMIMOv3.generate_data(parameters)
    print(f"Scenes {chunk[0]}–{chunk[-1]} generation time: {time.time() - start:.2f}s")

    # combine all_data or save in the Disk
    all_data.extend(data_chunk)

    # free memory 
    del data_chunk
    gc.collect()

# comvine Dataset
dataset = all_data


print(parameters['user_rows'])

## About Information
User : 737
UE antenna : 1
BS antenna : 32  Shape(a+bj)
subcarrier : 64

In [ ]:
# Unmasked Data Model(gru
# separate maksed data and unmasked data

## Data Preprocessing

In [ ]:
# ─────────────────────────────────────────────
# ❶ IterableDataset: stream all users and subcarriers
import torch
from torch.utils.data import IterableDataset, DataLoader
import numpy as np

class UnmaskedChannelSeqDataset(IterableDataset):
    """
    • Predict next-timestep vector from seq_len past channel vectors (real 64 + imag 64 → 128)
    • Vectors are power-normalized to unit average power before returning
    """
    def __init__(self, scenes, seq_len: int = 5, eps: float = 1e-9):
        super().__init__()
        self.scenes   = scenes
        self.seq_len  = seq_len
        self.eps      = eps                        # small constant to prevent division by zero
        ch0           = scenes[0][0]['user']['channel']
        self.U        = ch0.shape[0]               # number of users
        self.A        = ch0.shape[2]               # number of antennas (e.g., 32)
        self.S        = ch0.shape[3]               # number of subcarriers (e.g., 64)
        self.vec_len  = 2 * self.A                 # length of real+imag vector (2 * antennas)
        0

    def _vec(self, scene, u: int, sc: int) -> torch.Tensor:
        """Convert complex channel vector (32,) to float32 vector (64,) + power normalization"""
        h = scene[0]['user']['channel'][u, 0, :, sc]          # complex vector of shape (A,)
        v = np.concatenate([h.real, h.imag]).astype(np.float32)
        p = np.mean(v * v) + self.eps                         # average power: mean of squared amplitudes
        v /= np.sqrt(p)                                       # normalize to unit power
        return torch.from_numpy(v)                            # return tensor of shape (vec_len,)

    def __iter__(self):
        T = len(self.scenes)
        for t in range(self.seq_len, T):  # target timestep index
            past_scenes = self.scenes[t - self.seq_len : t]
            tgt_scene   = self.scenes[t]
            for u in range(self.U):
                for s in range(self.S):
                    seq = torch.stack([self._vec(ps, u, s) for ps in past_scenes])
                    if not torch.any(seq):        # skip if all sequence entries are zero
                        continue
                    target = self._vec(tgt_scene, u, s)
                    if not torch.any(target):     # skip if target vector is zero
                        continue
                    yield seq, target             # masked_pos removed here
                # shapes: (seq_len, vec_len) / (vec_len,)

    def __len__(self):
         return (len(self.scenes) - self.seq_len) * self.U * self.S
# ─────────────────────────────────────────────


In [ ]:
# ─────────────────────────────────────────────
# ❶ IterableDataset: stream all users and subcarriers
import torch
from torch.utils.data import IterableDataset, DataLoader
import numpy as np

class MaskedChannelSeqDataset(IterableDataset):
    """
    • Predict next-timestep vector from seq_len past channel vectors (real 64 + imag 64 → 128)
    • Vectors are power-normalized to unit average power before returning
    """
    def __init__(self, scenes, seq_len: int = 5, eps: float = 1e-9):
        super().__init__()
        self.scenes   = scenes
        self.seq_len  = seq_len
        self.eps      = eps                        # small constant to prevent division by zero
        ch0           = scenes[0][0]['user']['channel']
        self.U        = ch0.shape[0]               # number of users
        self.A        = ch0.shape[2]               # number of antennas (e.g., 32)
        self.S        = ch0.shape[3]               # number of subcarriers (e.g., 64)
        self.vec_len  = 2 * self.A                 # length of real+imag vector (2 * antennas)
        0

    def _vec(self, scene, u: int, sc: int) -> torch.Tensor:
        """Convert complex channel vector (32,) to float32 vector (64,) + power normalization"""
        h = scene[0]['user']['channel'][u, 0, :, sc]          # complex vector of shape (A,)
        v = np.concatenate([h.real, h.imag]).astype(np.float32)
        p = np.mean(v * v) + self.eps                         # average power: mean of squared amplitudes
        v /= np.sqrt(p)                                       # normalize to unit power
        return torch.from_numpy(v)                            # return tensor of shape (vec_len,)

    def __iter__(self):
        T = len(self.scenes)
        for t in range(self.seq_len, T):  # target timestep index
            past_scenes = self.scenes[t - self.seq_len : t]
            tgt_scene   = self.scenes[t]
            for u in range(self.U):
                for s in range(self.S):
                    seq = torch.stack([self._vec(ps, u, s) for ps in past_scenes])
                    if not torch.any(seq):        # skip if all sequence entries are zero
                        continue
                    target = self._vec(tgt_scene, u, s)
                    if not torch.any(target):     # skip if target vector is zero
                        continue
                    yield seq, target             # masked_pos removed here
                # shapes: (seq_len, vec_len) / (vec_len,)

    def __len__(self):
         return (len(self.scenes) - self.seq_len) * self.U * self.S
# ─────────────────────────────────────────────


## Split Train/Val

In [ ]:
# ❷ Train/Validation DataLoader split train : val = 6 : 4
seq_len      = 5
split_ratio  = 0.6
split_idx    = int(len(dataset) * split_ratio)

unmasked_train_ds = UnmaskedChannelSeqDataset(dataset[:split_idx], seq_len=seq_len)
unmasked_val_ds   = UnmaskedChannelSeqDataset(dataset[split_idx:], seq_len=seq_len)

# iterate over train_ds to compute min and max of features/targets

batch_size   = 32
unmasked_train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
unmasked_val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)
# ─────────────────────────────────────────────


## Normalization Dataset(Min-Max)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data      import TensorDataset, DataLoader
import numpy as np
import torch

# 1) extract Numpy 
# tgt = target
X_list, y_list = [], []
for seq, tgt in ChannelSeqDataset(dataset, seq_len=seq_len):
    X_list.append(seq.numpy())
    y_list.append(tgt.numpy())

X = np.stack(X_list, axis=0)  # (N, L, D)
y = np.stack(y_list, axis=0)  # (N, D)

# 2) Train/Val split
split_idx = int(len(X) * split_ratio)
X_tr, X_va = X[:split_idx], X[split_idx:]
y_tr, y_va = y[:split_idx], y[split_idx:]

# 3) normalize MinMaxScaler
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
Ntr, L, D = X_tr.shape

scaler_x.fit(X_tr.reshape(-1, D))
scaler_y.fit(y_tr)

# 4) Restore original shape after transform
X_tr_s = scaler_x.transform(X_tr.reshape(-1, D)).reshape(Ntr, L, D)

Nva = X_va.shape[0]
X_va_s = scaler_x.transform(X_va.reshape(-1, D)).reshape(Nva, L, D)

y_tr_s = scaler_y.transform(y_tr)
y_va_s = scaler_y.transform(y_va)

# 5) Dataset & DataLoader
train_ds    = TensorDataset(
    torch.from_numpy(X_tr_s).float(),
    torch.from_numpy(y_tr_s).float()
)
val_ds      = TensorDataset(
    torch.from_numpy(X_va_s).float(),
    torch.from_numpy(y_va_s).float()
)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)

# 6) first batch unpacking
seqs, tgts = next(iter(train_loader))  # seqs: (B, L, D), tgts: (B, D)